## Token Classification. Практическое задание (PJ) Граур Андрей Константинович

Для закрепления материала модуля вам необходимо решить задачу NER для предоставленного [датасета](https://lms.skillfactory.ru/asset-v1:SkillFactory+MFTIDS+SEP2023+type@asset+block@FactRuEval.zip), используя любые доступные вам средства. Модель должна обучаться на файле `train.txt`, валидироваться на файле `dev.txt`, а её качество необходимо оценить на файле `test.txt`.
Для достижения наилучшего результата уделите внимание подбору гиперпарметров как в плане архитектуры, так и в плане обучения модели.

Критерии оценивания проекта:
- общее качество кода и следование PEP-8;
- использование рекуррентных сетей;
- использованы варианты архитектур, близкие к state of the art для данной задачи;
- произведен подбор гиперпараметров;
- использованы техники изменения learning rate (lr scheduler);
- использована адекватная задаче функция потерь;
- использованы техники регуляризации;
- корректно проведена валидация модели;
- использованы техники ensemble;
- использованы дополнительные данные;
- итоговое значение метрики качества > 0.6 (f1)

## 1) Импортирую необхоимдые мне зависимости

In [ ]:
import numpy as np
import pandas as pd
import json
import collections
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(1)
from matplotlib import pyplot as plt
import collections
from sklearn.metrics import classification_report

## 2) Достаю данные

In [ ]:
path = '/content/data.zip'
! wget https://lms.skillfactory.ru/asset-v1:SkillFactory+MFTIDS+SEP2023+type@asset+block@FactRuEval.zip -O {path}


--2023-10-04 08:54:42--  https://lms.skillfactory.ru/asset-v1:SkillFactory+MFTIDS+SEP2023+type@asset+block@FactRuEval.zip
Resolving lms.skillfactory.ru (lms.skillfactory.ru)... 51.250.7.2
Connecting to lms.skillfactory.ru (lms.skillfactory.ru)|51.250.7.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 870425 (850K) [application/x-zip-compressed]
Saving to: ‘/content/data.zip’

/content/data.zip   100%[===================>] 850.02K  1.02MB/s    in 0.8s    

2023-10-04 08:54:44 (1.02 MB/s) - ‘/content/data.zip’ saved [870425/870425]



In [ ]:
! unzip {path}

Archive:  /content/data.zip
  inflating: dev.txt                 
  inflating: test.txt                
  inflating: train.txt               


In [ ]:
# Функция для преобразования текстовго файла в json формат
def json_mapper(path):
    save_path = f"{path.split('.')[0]}"+'.json'
    res = dict()
    with open(path, 'r') as in_file:
      stripped = (line.strip('\n') for line in in_file)
      line_dict = dict()
      txt = []
      label_ = []
      j = 0
      for i, line in enumerate(stripped):
        if line:
          token_, line_ =  line.split(' ')
          txt.append(token_)
          label_.append(line_)
        if not line:
          line_dict['token'] = txt
          line_dict['ner'] = label_
          res.update({j: line_dict})
          txt = []
          label_ = []
          line_dict = dict()
          j+=1

    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(res, f, ensure_ascii=False, indent=4)

In [ ]:
json_mapper('train.txt')
json_mapper('dev.txt')
json_mapper('test.txt')

## 3) Проверяю и подготавливаю данные

In [ ]:
# Read JSON file
def read_json(path):
  with open(path) as data_file:
      return json.load(data_file)
    
train_ds = read_json('train.json')
valid_ds = read_json('dev.json')
test_ds = read_json('test.json')

Эта функция фильтрует последовательности входных данных по их длине. Она принимает список последовательностей sequences и необязательный аргумент max_length, который указывает максимальную длину последовательности.

In [ ]:
def filter_ds(sequences, max_length=256):
    lengths = [len(sequences[str(i)]['token']) for i in range(len(sequences))]
    print(f'Maximum length: {max(lengths)}')
    print(f'Minimum length: {min(lengths)}')
    print(f'Average length: {sum(lengths)/len(lengths)}')

    short_sequences = []
    for i in range(len(sequences)):
      seq = sequences[str(i)]
      if len(seq['token']) <= max_length:
        short_sequences.append(seq)

    print(f'% of short sequences: {100 * len(short_sequences)/len(sequences)}')

    X = [[c for c in x['token']] for x in short_sequences] #[' '.join(c for c in x['token']) for x in short_sequences] #
    y = [[c for c in y['ner']] for y in short_sequences]
    lengths = [len(x) for x in X]
    print(f'Maximum cleared length: {max(lengths)}')
    return X, y

short_sequences = []
for i in range(len(train_ds)):
  seq = train_ds[str(i)]
  if len(seq['token']) < 256:
    short_sequences.append(seq)

In [ ]:
max_length = 200
X_train, y_train = filter_ds(train_ds, max_length)
print()
X_valid, y_valid = filter_ds(valid_ds, max_length)
print()
X_test, y_test = filter_ds(test_ds, max_length)

Maximum length: 103
Minimum length: 1
Average length: 20.2285050348567
% of short sequences: 100.0
Maximum cleared length: 103

Maximum length: 207
Minimum length: 2
Average length: 19.824167312161116
% of short sequences: 99.96127033307513
Maximum cleared length: 150

Maximum length: 222
Minimum length: 2
Average length: 20.541440743609606
% of short sequences: 99.96127033307513
Maximum cleared length: 116


## 3) Произвожу encoding

Здесь создаю словари для преобразования токенов и меток в соответствующие индексы, используемые для обучения модели машинного обучения.

In [ ]:
minimal_frequency = 2
train_and_valid_ds = X_train + X_valid
func = (token for sequence in train_and_valid_ds for token in sequence)
_token = ["{unk}"] + [token for token, count in collections.Counter(func).items() if count >= minimal_frequency]
_index = collections.defaultdict(lambda: 1, {token: index for index, token in enumerate(_token)})
_label = list(set([label for target in y_train for label in target]))
label_with_index = {label: index for index, label in enumerate(_label)}
_label

['B-LOC', 'B-PER', 'B-ORG', 'I-LOC', 'O', 'I-PER', 'I-ORG']

In [ ]:
label_with_index

{'B-LOC': 0,
 'B-PER': 1,
 'B-ORG': 2,
 'I-LOC': 3,
 'O': 4,
 'I-PER': 5,
 'I-ORG': 6}

In [ ]:
max_length = max([len(x) for x in train_and_valid_ds])
NUM_CLASSES = len(label_with_index)
print('Text vocabulary size: ', len(_index))
print('Label vocabulary size: ', NUM_CLASSES)
print('Maximum sequence length: ', max_sequence_length)

Text vocabulary size:  43026
Label vocabulary size:  7
Maximum sequence length:  150


In [ ]:
X_enc_train = [[_index[token] for token in sequence] for sequence in X_train]
y_enc_train = [[label_with_index[label] for label in target] for target in y_train]
X_enc_valid = [[_index[token] for token in sequence] for sequence in X_valid]
y_enc_valid = [[label_with_index[label] for label in target] for target in y_valid]
X_enc_test = [[_index[token] for token in sequence] for sequence in X_test]
y_enc_test = [[_index[label] for label in target] for target in y_test]

## 4) Строю модель

Сначала произвожу вынесение параметров в отдельные переменные. Так же для обучения планирую использовать свою ПК с видеокартой NVIDIA GeForce GTX 1660 с поддержкой cuda. Для этого были установлены все необходимые библиотека и настроено окружение в Anakonda.

In [ ]:
vocabulary_size = len(token2index)
tags_size = NUM_CLASSES
epochs = 5
embedding_size = 128
hidden_size = 128
batch_size = 2048

def prepare_sequence(seq):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    return torch.tensor(seq, dtype=torch.long).to(device)

prepare_sequence(X_enc_valid[0])

tensor([195, 395, 396, 397, 398, 399,  33, 400, 248,   8, 401, 402, 403, 404,
        405, 406, 248,  36])

Здесь определен класс `LSTMTagger`, который является моделью для тегирования последовательностей с использованием LSTM. В конструкторе класса определены слои и параметры модели, включая вложения слов, LSTM слой и линейный слой для преобразования скрытого состояния в прогнозируемое пространство меток. Метод `forward` выполняет прямой проход модели, принимая на вход последовательность слов и возвращая пространство меток. Метод `predict` использует модель для предсказания меток для заданной последовательности слов.

In [ ]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size,
                                            embedding_dim) 
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.hidden2tag = nn.Linear(2 * hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        return (autograd.Variable(torch.zeros(2, 1, self.hidden_dim).to(device)),
                autograd.Variable(torch.zeros(2, 1, self.hidden_dim).to(device)))  

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        return tag_space

    def predict(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        logits = self.hidden2tag(lstm_out.view(len(sentence), -1))
        pred = torch.softmax(logits, dim=1)
        pred = torch.argmax(pred, dim=1)
        return pred

In [ ]:
model = LSTMTagger(embedding_size,hidden_size,vocabulary_size,tags_size)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

LSTMTagger(
  (word_embeddings): Embedding(35630, 128)
  (lstm): LSTM(128, 128, bidirectional=True)
  (hidden2tag): Linear(in_features=256, out_features=7, bias=True)
)

## 5) Тренирую модель

### 5.1) Подготовка
Функция `eval(X_dev, y_dev)` используется для оценки модели на тестовых данных. Внутри функции происходит следующее:

- Инициализируются переменные `sum_loss`, `good` и `bad` для подсчета общей потери, количества правильных и неправильных предсказаний.
- Модель переводится в режим оценки с помощью метода `eval()`.
- Для каждой пары входных данных `sentence_in` и меток `targets` из тестовых данных происходит следующее:
  - Сбрасываются градиенты и скрытое состояние модели.
  - Выполняется прямой проход модели для получения логитов.
  - Вычисляется потеря с использованием функции потерь `loss_function`.
  - Применяется функция `softmax` к логитам и выполняется предсказание меток с помощью функции `argmax`.
  - Обновляются суммарную потерю, количество правильных и неправильных предсказаний.
- Возвращаются средняя потеря и точность модели на тестовых данных.

In [ ]:
def eval(X_dev,y_dev):
    sum_loss = 0.0
    good = 0.0
    bad = 0.0
    model.eval()
    for sentence_in, targets in zip(X_dev,y_dev):
      model.hidden = model.init_hidden()
      logits = model(prepare_sequence(sentence_in))
      targets = prepare_sequence(targets)
      loss = loss_function(logits, targets)
      preds = torch.softmax(logits, dim=1)
      preds = torch.argmax(preds, dim=1)
      sum_loss += loss.item()
      correct = (preds == targets).sum().item()
      good += correct
      bad  += len(targets) - correct
    return sum_loss/len(X_dev), good / (good + bad)

### 5.3) Непосредственно обучение

Здесь происходит обучение модели на тренировочных данных с использованием метода стохастического градиентного спуска.

- `eval_number` - определяет, через сколько итераций происходит оценка точности и потери на валидационных данных.
- `loss_function` - функция потерь для вычисления потери модели.
- `optimizer` - оптимизатор, используемый для обновления параметров модели.
- `loss_history`, `dev_history`, `dev_acc_history` - списки для сохранения истории потери на тренировочных данных, потери на валидационных данных и точности на валидационных данных соответственно.
- `sum_loss` - переменная для накопления потери на каждой итерации.
- `reduce_train_size` - коэффициент, определяющий размер тренировочных данных.
- `stop` - количество итераций, после которого обучение останавливается.
- Внешний цикл `for epoch in range(epochs)` выполняется для каждой эпохи обучения.
- Внутренний цикл `for sentence_in, targets in zip(X_enc_train[:stop], y_enc_train[:stop])` выполняется для каждой пары входных данных и меток из тренировочных данных.
- Проверяется, достигнуто ли количество итераций, чтобы оценить точность и потерю на валидационных данных и сохранить результаты.
- Градиенты обнуляются, скрытое состояние модели сбрасывается.
- Выполняется прямой проход модели для получения предсказаний.
- Вычисляется потеря с использованием функции потерь.
- Обновляется суммарная потеря.
- Выполняется обратное распространение градиентов и обновление параметров модели с помощью оптимизатора.

In [ ]:
eval_number = 1000
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
dev_history = []
loss_history = []
dev_acc_history = []
sum_loss = 0.0
reduce_train_size = 2
stop = int(len(X_enc_train)/reduce_train_size)

for epoch in range(epochs):
    i = 0
for epoch in range(epochs):
    print(f'\nTrain on epoch {epoch+1}')
    for sentence_in, targets in zip(X_enc_train[:stop], y_enc_train[:stop]):
        i+=1
        if (i % eval_number == 0):
            t = 100 * i/(epochs*stop)
            loss_history.append(sum_loss / eval_number)
            dev_loss, dev_acc = eval(X_enc_valid, y_enc_valid)
            dev_history.append(dev_loss)
            dev_acc_history.append(dev_acc)
            print(f'{t:.2f}% train with loss {sum_loss / eval_number:.4f} |   val_loss {dev_loss:.4f}, val_acc {dev_acc:.4f}')
            sum_loss = 0.0
        model.zero_grad()
        #model.train()
        model.hidden = model.init_hidden()
        tag_scores = model(prepare_sequence(sentence_in))
        targets = prepare_sequence(targets)
        loss = loss_function(tag_scores, targets)
        sum_loss += loss.item()
        loss.backward()
        optimizer.step()


Train on epoch 1
5.16% train with loss 0.6121 |   val_loss 0.5640, val_acc 0.8503
10.33% train with loss 0.5549 |   val_loss 0.5132, val_acc 0.8590
15.49% train with loss 0.4859 |   val_loss 0.4577, val_acc 0.8688

Train on epoch 2
20.66% train with loss 0.4299 |   val_loss 0.4233, val_acc 0.8761
25.82% train with loss 0.4007 |   val_loss 0.4126, val_acc 0.8811
30.98% train with loss 0.3610 |   val_loss 0.3753, val_acc 0.8870
36.15% train with loss 0.3241 |   val_loss 0.3568, val_acc 0.8930

Train on epoch 3
41.31% train with loss 0.3033 |   val_loss 0.3343, val_acc 0.8971
46.48% train with loss 0.2851 |   val_loss 0.3221, val_acc 0.9007
51.64% train with loss 0.2428 |   val_loss 0.3109, val_acc 0.9068
56.80% train with loss 0.2187 |   val_loss 0.2926, val_acc 0.9115

Train on epoch 4
61.97% train with loss 0.2069 |   val_loss 0.2866, val_acc 0.9119
67.13% train with loss 0.1922 |   val_loss 0.3144, val_acc 0.8996
72.30% train with loss 0.1651 |   val_loss 0.2864, val_acc 0.9081
77.46

## 6) Тестирую 


In [ ]:
test_loss, test_acc = eval(X_enc_test, y_enc_test)
test_loss, test_acc

(0.24495130375448473, 0.9242464404725841)

In [ ]:
preds = []
for sentence_in in X_enc_test:
    model.eval()
    model.hidden = model.init_hidden()
    logits = model(prepare_sequence(sentence_in))
    targets = prepare_sequence(targets)
    pred = torch.softmax(logits, dim=1)
    pred = torch.argmax(pred, dim=1)
    preds.append(pred.cpu().numpy())

<ipython-input-119-039af80a889d>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(seq, dtype=torch.long).to(device)


In [ ]:
correct  = 0
incorrect = 0
for i in range(len(X_enc_test)):
  y_pred = preds[i]
  y_true = np.array(y_enc_test[i])
  correct += np.nonzero(y_pred == y_true)[0].shape[0]
  incorrect += np.nonzero(y_pred != y_true)[0].shape[0]

print("Correct predicted classes:", correct)
print("Incorrect predicted classes:", incorrect)
print(correct / (correct + incorrect))

Correct predicted classes: 48815
Incorrect predicted classes: 4001
0.9242464404725841


## 7) Вывод

Я использовал модель LSTMTagger для тегирования последовательностей. Модель состоит из вложений слов, двунаправленного LSTM слоя и линейного слоя для преобразования скрытого состояния в пространство меток. 
Во время обучения модели используется стохастический градиентный спуск с оптимизатором SGD и функцией потерь CrossEntropyLoss. Обучение происходит в 5 эпох, где каждая эпоха состоит из нескольких итераций по батчам тренировочных данных. 
Каждые eval_number итераций происходит оценка точности и потери модели на валидационных данных. После каждой оценки результаты сохраняются в соответствующих списках.

In [ ]:
def flattering(sequences):
  res = []
  for seq in sequences:
    res.extend(seq)
  return res

print(classification_report(flattering(y_enc_test), flattering(preds), target_names=_label))

              precision    recall  f1-score   support

       B-LOC       0.84      0.75      0.79      1508
       B-PER       0.64      0.67      0.65      2132
       B-ORG       0.78      0.60      0.68      1734
       I-LOC       0.88      0.65      0.75       342
           O       0.95      0.97      0.96     44706
       I-PER       0.76      0.70      0.73      1304
       I-ORG       0.68      0.48      0.57      1090

    accuracy                           0.92     52816
   macro avg       0.79      0.69      0.73     52816
weighted avg       0.92      0.92      0.92     52816

